In [2]:
#Imports
import pandas as pd
import pprint as pprint
import re
import numpy as np

#Notes for K.  
# 1. In combined_dis_df I've simplified the column names.  I observed your VSC terminal was autofilling these in.  Let's observe how it reacts to
#.   this change?  Does it automatically update the code you've written in main that references the old column names?  We can revert this change 
#.   simply, by commenting the column name changes out.

#ToDos
# 1. Write output .csv for combined disengagement reports

#More Complex ToDos
#1.  Restructure these things as a function, which can pass data to main?  Or is it better for these things to produce an output csv that main can
#.    then read?  Does size matter for this sort of thing?  Like, with 20k rows etc, is there a best practices?

# Notes to recall a/b data
# 1. There is no 2022 driverless disengagement report.  2020-21 and 2023 combined are 18 rows.  Does this seem strange?  
# 2. If we accept the driverless disengagement at face value, there were startingly few disengagements in NVO mode.  How does this correlate to NVO miles?
# 3. There are only 6 disengagement reports for "Parking Facilities".  Given the amount of testing that (presumably) occured in parking lots
#.  And parking Garages, I'm inclined to think that these generally didn't result in reports to the state.  
# 4.  There are 904 distinct reasons for disengagement.  This disengagement report has 17.7k items.  
#.    The top 10 of these account for 10k of the descriptions.  The top 3 account for 5.8k.  The top 5, 7.8k
# 5.  There's no way Cruise had so few disengagements

'''
This is the code I used to poke at the values within the Disengagement Description
vc = combined_dis_df['Disengagement Description'].value_counts()
counter = 0
for i in range(10):
    print(vc[i])
    counter += vc[i]
print()
print(counter)
'''

#Questions
# 1. I am unsure of when to use inplace=True .  When is appropriate?

"\nThis is the code I used to poke at the values within the Disengagement Description\nvc = combined_dis_df['Disengagement Description'].value_counts()\ncounter = 0\nfor i in range(10):\n    print(vc[i])\n    counter += vc[i]\nprint()\nprint(counter)\n"

In [3]:
#Lists used by K to assist read in.  Monthly list used for simplified mileage reports
years = ["2020", "2021", "2022", "2023"]
years = sorted(list(years))

months = ["DEC-1", "JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JULY", "AUG", "SEP", "OCT", "NOV"]

In [4]:
## Read in disengagements data, dropping null rows & columns
# Yearly disengagement 
# These are all reading in correctly

dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV.csv", encoding='utf-8')
    # Allows the search to be broad
    except FileNotFoundError:
        #print(f"Type 1 File for year {year} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV.csv", encoding='latin-1')
    dis_dfs.append(annual_disengagements)
    
disengagement_df = pd.concat(dis_dfs, ignore_index=True)

# 1st time disengagement reports
first_time_dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-first-time-filers.csv", encoding='utf-8')
    except FileNotFoundError:
        #print(f"Type 2 File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-first-time-filers.csv", encoding='latin-1')
    first_time_dis_dfs.append(annual_disengagements)
    
first_df = pd.concat(first_time_dis_dfs, ignore_index=True)

# Driverless reports.  2022 is missing.  There are only 2 files here.  
# I've combined the above 2x for loops to snag them both,since the files have different naming conventions
driverless_dis_dfs = []
for year in years:
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='utf-8')
    except FileNotFoundError:
        #print(f"Type 3a File for year {year}-{str(int(year[3]) + 21)} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-{str(int(year[3]) + 21)}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='latin-1')
    driverless_dis_dfs.append(annual_disengagements)

for year in years:    
    try:
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='utf-8')
        # Allows the search to be broad
    except FileNotFoundError:
        #print(f"Type 3b File for year {year} not found. Skipping...")
        continue
        # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_disengagements = pd.read_csv(f"data/Disengagements/{year}-Autonomous-Vehicle-Disengagement-Reports-CSV-driverless.csv", encoding='latin-1')
    driverless_dis_dfs.append(annual_disengagements)

autonomous_dis_df = pd.concat(driverless_dis_dfs, ignore_index=True)


In [5]:
## Combining 3x disengagement DFs into a single DF.  They all have same column names, withh one report having an additional column.
# 3x active DFs for the disengagement info are
# disengagement_df
# first_df
# autonomous_dis_df

disengagement_dfs_list = [disengagement_df, first_df, autonomous_dis_df]
combined_dis_df = pd.concat(disengagement_dfs_list, ignore_index=True)

In [6]:
## Cleaning combined_dis_DF

# The unnamed column 10 only appears in the 2021 report, and the 71 entries are all for IMAGRY INC., reading "Perception" or "Planning"
# Dropped Column
# Try is used to allow the block to be re-run without generating an error, or the need to rerun above blocks

try:
    combined_dis_df = combined_dis_df.drop('Unnamed: 9', axis=1)
except KeyError:
    print("Column 'Unnamed: 9' already dropped")

# Dropped 9k+ null rows
combined_dis_df.dropna(axis=0, how='any', inplace=True)

# Updating column names
new_combined_dis_df_column_names = ['Company', 'Permit #', 'Date', 'VIN', 'Driverless Capable', 'Driver Present', 'Disengagement Initiation', 
                                    'Disengagement Location', 'Disengagement Description']
combined_dis_df.columns = new_combined_dis_df_column_names

# Dropping time from DATE column (some limited % of entries include this), converting to_datetime
# Could these further be combined into one line?  Where can I learn more a/b this?
combined_dis_df['Date'] = combined_dis_df['Date'].astype(str).str.split(" ", expand=True)[0]
combined_dis_df['Date'] = pd.to_datetime(combined_dis_df['Date'], errors='raise', dayfirst=True, format='mixed')



In [7]:
### Cleaning up combined_dis_df
## Unifying text format, all except last column (descriptions), and skipping over Date column via try/except
# Strip any spaces at the start or end, .upper, 
for i in combined_dis_df.iloc[:,:-1]:
    #print("Column stripped & upper: " + i)
    try:
        combined_dis_df[i] = combined_dis_df[i].str.strip()
        combined_dis_df[i]= combined_dis_df[i].str.upper()
    except AttributeError:
        continue

#Cleaning 'Disengagement Initiation
combined_dis_df['Disengagement Initiation'] = combined_dis_df['Disengagement Initiation'].str.replace(r'^TEST DRIVER - SOFT STOP$', 'TEST DRIVER', regex=True)
combined_dis_df['Disengagement Initiation'] = combined_dis_df['Disengagement Initiation'].str.replace(r'^AV SYSTEM - EMERGENCY STOP$', 'AV SYSTEM', regex=True)
combined_dis_df['Disengagement Initiation'] = combined_dis_df['Disengagement Initiation'].str.replace(r'^TEST DRIVE$', 'TEST DRIVER', regex=True)
combined_dis_df['Disengagement Initiation'] = combined_dis_df['Disengagement Initiation'].str.replace(r'^DRIVER$', 'TEST DRIVER', regex=True)
combined_dis_df['Disengagement Initiation'] = combined_dis_df['Disengagement Initiation'].str.replace(r'^SOFTWARE$', 'AV SYSTEM', regex=True)
combined_dis_df['Disengagement Initiation'] = combined_dis_df['Disengagement Initiation'].str.replace(r'^ADS$', 'AV SYSTEM', regex=True)
combined_dis_df['Disengagement Initiation'] = combined_dis_df['Disengagement Initiation'].str.replace(r'^OPERATOR$', 'TEST DRIVER', regex=True)
combined_dis_df['Disengagement Initiation'].value_counts()

## Cleaning 'Company'
# Dropping all words after the first word in 'Company', dropping all commas left floating at the end, capitlizing all words.
combined_dis_df["Company"] = combined_dis_df["Company"].str.upper().str.split(" ", expand=True)[0].str.replace(",", "")

## Cleaning 'Disengagement Location'
# Freeway has a technical definition (it's free) vs Toll Road (paid).  
# Highway usually references state funded 'interstate' style roads, whereas Interstates are federally funded
# Express-Way is a subset of Toll Roads, referencing less exits and segregated lanes.
# Regardless, I've combined these all into "Highway" so the contrast b/t 'Street' is clear.  These different types of high traffick roads are
#   a mish-mash in the Bay Area and LA, and nothing indicates these distinctions were used with percision (colliqually they are almost never) or that
#   even if they were, it would matter to our analysis
# Also combined "Urban" into "Street"
combined_dis_df['Disengagement Location'] = combined_dis_df['Disengagement Location'].str.upper()
combined_dis_df['Disengagement Location'] = combined_dis_df['Disengagement Location'].str.replace("FREEWAY ", "FREEWAY")
combined_dis_df['Disengagement Location'] = combined_dis_df['Disengagement Location'].str.replace("FREEWAY", "HIGHWAY")
combined_dis_df['Disengagement Location'] = combined_dis_df['Disengagement Location'].str.replace("EXPRESS WAY", "HIGHWAY")
combined_dis_df['Disengagement Location'] = combined_dis_df['Disengagement Location'].str.replace("INTERSTATE", "HIGHWAY")
combined_dis_df['Disengagement Location'] = combined_dis_df['Disengagement Location'].str.replace("URBAN", "STREET")

## Cleaning up a tricky problem in VINs.  See below cell for code used to ID and isolate problem
#This code captures all the unique 5 letter vins, drops the ~, 
unique_vin_4_letter_list = set(list(combined_dis_df.loc[combined_dis_df['VIN'].str.contains('~')]['VIN'].str.replace("~", "")))
zoox_vin_17_character_list = set(list(combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.len() == 17)]['VIN']))

#This nested for loop checks to see which nnnn's are a subset of a full VIN, and then replaces all instances with that nnnn w/ the full VIN
for i in unique_vin_4_letter_list:
    for j in zoox_vin_17_character_list:
        if str(i) in str(j):
            combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.contains(i)), 'VIN'] = j
            #print(combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.contains(i))]['VIN'])

# This code confirms 2 of the unique ~nnnn VINs have been replaced.  
# Length of 19 prints, instead of 21 in original set(list) .  23 values in in total entry list, instead of 25

In [8]:
'''
## Data Quality inspection post-cleanup.  All nice and pretty.  Remove ''' ''' to run and confirm clean data.
# VIN Cleanup Status
vin_length_check = set(list(combined_dis_df['VIN'].str.len().sort_values()))
print(f"VIN contains strings of length: {vin_length_check}")
vin_5_letter_list = combined_dis_df.loc[combined_dis_df['VIN'].str.len() == 5, 'VIN']
#[vin for vin in vin_list if len(vin)==5]
vin_18_letter_list = combined_dis_df.loc[combined_dis_df['VIN'].str.len() == 18, 'VIN']
print(f"There are {len(vin_5_letter_list)} 5 letter VINs. \n There are {len(set(list(vin_5_letter_list)))} unique 5 letter VINs. \n They are {(set(list(vin_5_letter_list)))}")
print()
print(f"There are {len(vin_18_letter_list)} 18 letter VINs. \n There are {len(set(list(vin_18_letter_list)))} unique 18 letter VINs. \n They are {set(list(vin_18_letter_list))}")
print("\n Space Break \n")

# Review all columns value_counts, verify they're cleaned
for i in combined_dis_df:
    print(combined_dis_df[i].value_counts())
    print("\n SPACE BREAK \n")
'''

'\n## Data Quality inspection post-cleanup.  All nice and pretty.  Remove  to run and confirm clean data.\n# VIN Cleanup Status\nvin_length_check = set(list(combined_dis_df[\'VIN\'].str.len().sort_values()))\nprint(f"VIN contains strings of length: {vin_length_check}")\nvin_5_letter_list = combined_dis_df.loc[combined_dis_df[\'VIN\'].str.len() == 5, \'VIN\']\n#[vin for vin in vin_list if len(vin)==5]\nvin_18_letter_list = combined_dis_df.loc[combined_dis_df[\'VIN\'].str.len() == 18, \'VIN\']\nprint(f"There are {len(vin_5_letter_list)} 5 letter VINs. \n There are {len(set(list(vin_5_letter_list)))} unique 5 letter VINs. \n They are {(set(list(vin_5_letter_list)))}")\nprint()\nprint(f"There are {len(vin_18_letter_list)} 18 letter VINs. \n There are {len(set(list(vin_18_letter_list)))} unique 18 letter VINs. \n They are {set(list(vin_18_letter_list))}")\nprint("\n Space Break \n")\n\n# Review all columns value_counts, verify they\'re cleaned\nfor i in combined_dis_df:\n    print(combined_

In [18]:
### MILEAGE REPORTS

## K's original mile_dfs readin AND mileage_df .  Leaving here so as to not throw off any of K's charts in main.
mile_dfs = []
for year in years:
    try:
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='utf-8')
    # Allows the search to be broad
    except FileNotFoundError:
        #print(f"File for year {year} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='latin-1')
    annual_mileage["Year"] = year
    annual_mileage.rename(columns={'ANNUAL TOTAL': 'Mileage Total'}, inplace=True)
    for i, month in enumerate(months, start=4):
        annual_mileage.columns.values[i] = month
    mile_dfs.append(annual_mileage)
    # INFO: each annual report has 22 cols

mileage_df = pd.concat(mile_dfs, ignore_index=True)
# mileage_df.set_index('VIN NUMBER', inplace=True)
#mileage_df['VIN NUMBER'].value_counts()

    
## New version, designed for joining to create multi-year M/m columns.  JULY is now JUL
new_mile_dfs = []
for year in years:
    try:
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='utf-8')
    # Allows the search to be broad
    except FileNotFoundError:
        #print(f"File for year {year} not found. Skipping...")
        continue
    # Got some kind of unicode error. This fixed it.
    except UnicodeDecodeError:
        # If utf-8 decoding fails, try another common encoding
        annual_mileage = pd.read_csv(f"data/Mileage/{year}-Autonomous-Mileage-Reports-CSV.csv", encoding='latin-1')
    annual_mileage.rename(columns={'Manufacturer': f"{year} Manufacturer"}, inplace=True)
    annual_mileage.rename(columns={'Permit Number': f"{year} Permit Number"}, inplace=True)
    
    #Dropping rows based on empty VIN, before changing name.  Can't get it to work dynamically
    # Took several tries, adding how and inplace, to make it stack
    annual_mileage.dropna(subset = ['VIN NUMBER'], how='any', inplace=True)
    print('Number of null values is: ' + str(annual_mileage['VIN NUMBER'].isnull().sum()))
    
    annual_mileage.rename(columns={'VIN NUMBER': f"{year} VIN"}, inplace=True)
    annual_mileage.rename(columns={'Annual Total of Disengagements': f"{year} Total Disengagements"}, inplace=True)
    for i, month in enumerate(months, start=4):
        annual_mileage.columns.values[i] = (f"{annual_mileage.columns.values[i][-4:]} {annual_mileage.columns.values[i][:3]}")
    annual_mileage.rename(columns={'ANNUAL TOTAL': f"{str(int(year) - 1)} DEC - {year} NOV Mileagle Total"}, inplace=True)
    unnamed_cols = ['Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21']
    annual_mileage= annual_mileage.drop(columns=unnamed_cols)
    new_mile_dfs.append(annual_mileage)
    #print(annual_mileage.columns.values)

#Want to learn how to do a join instead
new_mileage_df = pd.concat(new_mile_dfs, ignore_index=True)
for i in new_mile_dfs:
    print(i.shape)
print()
print(new_mileage_df.columns)

Number of null values is: 0
Number of null values is: 0
Number of null values is: 0
(1174, 17)
(1553, 17)
(1603, 17)

Index(['2021 Manufacturer', '2021 Permit Number', '2021 VIN',
       '2021 Total Disengagements', '2020 DEC', '2021 JAN', '2021 FEB',
       '2021 MAR', '2021 APR', '2021 MAY', '2021 JUN', '2021 JUL', '2021 AUG',
       '2021 SEP', '2021 OCT', '2021 NOV',
       '2020 DEC - 2021 NOV Mileagle Total', '2022 Manufacturer',
       '2022 Permit Number', '2022 VIN', '2022 Total Disengagements',
       '2021 DEC', '2022 JAN', '2022 FEB', '2022 MAR', '2022 APR', '2022 MAY',
       '2022 JUN', '2022 JUL', '2022 AUG', '2022 SEP', '2022 OCT', '2022 NOV',
       '2021 DEC - 2022 NOV Mileagle Total', '2023 Manufacturer',
       '2023 Permit Number', '2023 VIN', '2023 Total Disengagements',
       '2022 DEC', '2023 JAN', '2023 FEB', '2023 MAR', '2023 APR', '2023 MAY',
       '2023 JUN', '2023 JUL', '2023 AUG', '2023 SEP', '2023 OCT', '2023 NOV',
       '2022 DEC - 2023 NOV Mileagle T

In [24]:
#When these print checks are cleaned up, they can be reduced to a sentence or two of output that verifies no duplicate VINs in a year, etc

vin_check_placeholder = []
vins_2021 = pd.unique(new_mileage_df['2021 VIN'])                          
#vins_2022 = set(list(new_mileage_df.loc[new_mileage_df['2022 VIN']]))
#vins_2023 = set(list(new_mileage_df.loc[new_mileage_df['2023 VIN']]))

print(len(vins_2021))
#print(len(vins_2022))
#print(len(vins_2023))
print(vins_2021[:10])
print()
vins_check_placeholder= set(list(vins_2021))
print(len(vins_check_placeholder))
print()

vin_counter = 0
while vin_counter < 10:
    print(new_mileage_df[new_mileage_df.iloc[new_mileage_df['2021 VIN']==vins_2021[vin_counter].loc['2021 Manufacturer']]])
    vin_counter += 1

1174
['JTDKN3DU9A0059509' '4T1B21HK6KU514747' '3LN6L5LU2HR615153'
 '3LN6L2LU4GR629715' '3LN6L5LU1HR627987' '3LN6L2LU5GR607996'
 '2T2BGMCA5JC021381' '2T2BGMCA0GC007400' '2T2BGMCA0HC014753'
 '2T2BGMCA0HC016082']

1174



AttributeError: 'str' object has no attribute 'loc'

In [704]:
vin_col = ['2021 VIN', '2022 VIN', '2023 VIN']
for check in vin_col:
    non_null_count = new_mileage_df[check].notna().sum()
    print(f'Number of non values in {check}: {non_null_count}')
    
for i in new_mileage_df['2021 VIN']:
    if element['2021 VIN'].notna() == True:
        print(new_mileage_df.loc['2021 VIN', '2021 Manufacturer'])
    else:
        continue
   #Dataframe2[Dateframe2.Column2.isin([element])]

Number of non values in 2021 VIN: 1174
Number of non values in 2022 VIN: 1553
Number of non values in 2023 VIN: 1603


TypeError: string indices must be integers, not 'str'

In [692]:
new_mileage_df.loc[:,['2021 VIN', '2022 VIN', '2023 VIN']].tail()

,2021 VIN,2022 VIN,2023 VIN
4299,NaN,NaN,~1655
4300,NaN,NaN,~7590
4301,NaN,NaN,~4530
4302,NaN,NaN,~8547
4303,NaN,NaN,~9098
4304,NaN,NaN,~4787
4305,NaN,NaN,~2865
4306,NaN,NaN,~8508
4307,NaN,NaN,~6969
4308,NaN,NaN,~4153


In [656]:
#np.where(df['Level'] == 0, df['Item'], df['Item'].shift()
# Checking VINs to Manufacturer
# I think there might be an easier way to do this
#np.where(new_mileage_df.loc['2021 VIN']
         
#new_mileage_df.dtypes
# Going to have to break down and use a list and a for loop
#new_mileage_df.loc[new_mileage_df['2021 VIN'], '2021 Manufacturer']

vin_2021 = set(list(new_mileage_df.loc[new_mileage_df['2021 VIN'].notna()]))
len(vin_2021)

51

In [ ]:
non_null_count = mileage_df[check].notna()

unique_vin_4_letter_list = set(list(combined_dis_df.loc[combined_dis_df['VIN'].str.contains('~')]['VIN'].str.replace("~", "")))
zoox_vin_17_character_list = set(list(combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.len() == 17)]['VIN']))

#This nested for loop checks to see which nnnn's are a subset of a full VIN, and then replaces all instances with that nnnn w/ the full VIN
for i in unique_vin_4_letter_list:
    for j in zoox_vin_17_character_list:
        if str(i) in str(j):
            combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.contains(i)), 'VIN'] = j
            #print(combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.contains(i))]['VIN'])

In [ ]:
''' 
### Details on how I tracked down & issolated the VIN issue
## VINs should all be 17 letters long, per this link:  https://www.autocheck.com/vehiclehistory/vin-basics
#This line of code prints a preview, showing 3x VINs 18 characters long, and some unknown # of VINs 5 characters long
combined_dis_df['VIN'].str.len().sort_values()

#These lines ID that the column only contains elements 5, 17, or 18 characters long
vin_length_check = []
vin_length_check = set(list(combined_dis_df['VIN'].str.len().sort_values()))
print(vin_length_check)

#These lines sort them out.  The 18 letter VIN has 1 number too many at the end, and is repeated 3 times.  
vin_list = []
vin_list = combined_dis_df['VIN']
vin_5_letter_list = [vin for vin in vin_list if len(vin)==5]
vin_18_letter_list = [vin for vin in vin_list if len(vin)==18]
print(f"There are {len(vin_5_letter_list)} 5 letter VINs. \n There are {len(set(list(vin_5_letter_list)))} unique 5 letter VINs. \n They are {(set(list(vin_5_letter_list)))}")
print()
print(f"There are {len(vin_18_letter_list)} 18 letter VINs. \n There are {len(set(list(vin_18_letter_list)))} unique 18 letter VINs. \n They are {set(list(vin_18_letter_list))}")
print("\n Space Break \n")

#This searches the column for the first portion of the VIN, w/o the numbers at the end.  Nothing new turned up.  Same 3x entries.
# The 18 letter VINs - 3LN6L5LU6HR6267668 - is a Pony.AI car.  All 3 come from the 2021-Autonomous-Vehicle-Disengagement-Reports-CSV.csv
# The 5 letter VINs all begin with ~ .  They are all from ZOOX, 2022 and 2023 reports of the same name.
print(combined_dis_df.loc[combined_dis_df['VIN'].str.contains('3LN6L5LU6HR')])
print("\n SPACE BREAK \n")
print(combined_dis_df.loc[combined_dis_df['VIN'].str.contains('~')])

# Will keep an eye out for this 18 character VIN and its substrings in the mileage report, also the ~nnnn's

#This shows us all the ZOOX lines w/ VIN.  Some VINs are full, others not. Apx 40 total lines.  
# The ~nnnn that have a previous full entry can be inferred, but most don't have a previous entry
print(combined_dis_df.loc[combined_dis_df['Company'].str.contains('ZOOX'), ['Company', 'VIN']])

### THIS BELOW CODE IS IN USE
#This code captures all the unique 5 letter vins, drops the ~, 
unique_vin_4_letter_list = set(list(combined_dis_df.loc[combined_dis_df['VIN'].str.contains('~')]['VIN'].str.replace("~", "")))
zoox_vin_17_character_list = set(list(combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.len() == 17)]['VIN']))

#This nested for loop checks to see which nnnn's are a subset of a full VIN, and then replaces all instances with that nnnn w/ the full VIN
for i in unique_vin_4_letter_list:
    for j in zoox_vin_17_character_list:
        if str(i) in str(j):
            combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.contains(i)), 'VIN'] = j
            print(combined_dis_df.loc[(combined_dis_df["Company"] == 'ZOOX') & (combined_dis_df["VIN"].str.contains(i))]['VIN'])

# This code confirms 2 of the unique ~nnnn VINs have been replaced.  
# Length of 19 prints, instead of 21 in original set(list) .  23 values in in total entry list, instead of 25
### THIS ABOVE CODE IS IN USE

unique_vin_4_letter_list_CHECKER = set(list(combined_dis_df.loc[combined_dis_df['VIN'].str.contains('~')]['VIN']))
print(len(unique_vin_4_letter_list_CHECKER))
vin_list_post_cleanup = []
vin_list_post_cleanup = combined_dis_df['VIN']
vin_5_letter_list_post_cleanup = [vin for vin in vin_list_post_cleanup if len(vin)==5]
print(len(vin_5_letter_list))
'''